# Rf 6 0 4_Constraints
Likelihood and minimization: fitting with constraints



**Author:** 07/2008 - Wouter Verkerke  
<i><small>This notebook tutorial was automatically generated with <a href= "https://github.com/root-project/root/blob/master/documentation/doxygen/converttonotebook.py">ROOTBOOK-izer</a> from the macro found in the ROOT repository  on Thursday, August 29, 2019 at 02:58 AM.</small></i>

In [1]:
%%cpp -d
#include "RooRealVar.h"
#include "RooDataSet.h"
#include "RooGaussian.h"
#include "RooConstVar.h"
#include "RooPolynomial.h"
#include "RooAddPdf.h"
#include "RooProdPdf.h"
#include "RooFitResult.h"
#include "RooPlot.h"
#include "TCanvas.h"
#include "TAxis.h"
#include "TH1.h"

In [2]:
%%cpp -d
// This is a workaround to make sure the namespace is used inside functions
using namespace RooFit;

Create model and dataset
 ----------------------------------------------

Construct a gaussian p.d.f

In [3]:
RooRealVar x("x", "x", -10, 10);

RooRealVar m("m", "m", 0, -10, 10);
RooRealVar s("s", "s", 2, 0.1, 10);
RooGaussian gauss("gauss", "gauss(x,m,s)", x, m, s);


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt



Construct a flat p.d.f (polynomial of 0th order)

In [4]:
RooPolynomial poly("poly", "poly(x)", x);

Construct model = f*gauss + (1-f)*poly

In [5]:
RooRealVar f("f", "f", 0.5, 0., 1.);
RooAddPdf model("model", "model", RooArgSet(gauss, poly), f);

Generate small dataset for use in fitting below

In [6]:
RooDataSet *d = model.generate(x, 50);

Create constraint pdf
 -----------------------------------------

Construct gaussian constraint p.d.f on parameter f at 0.8 with resolution of 0.1

In [7]:
RooGaussian fconstraint("fconstraint", "fconstraint", f, RooConst(0.8), RooConst(0.2));

Method 1 - add internal constraint to model
 -------------------------------------------------------------------------------------

Multiply constraint term with regular p.d.f using rooprodpdf
 Specify in fitTo() that internal constraints on parameter f should be used

Multiply constraint with p.d.f

In [8]:
RooProdPdf modelc("modelc", "model with constraint", RooArgSet(model, fconstraint));

Fit model (without use of constraint term)

In [9]:
RooFitResult *r1 = model.fitTo(*d, Save());

[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
[#1] INFO:Minization --  The following expressions have been identified as constant and will be precalculated and cached: (poly)
[#1] INFO:Minization --  The following expressions will be evaluated in cache-and-track mode: (gauss)
 **********
 **    1 **SET PRINT           1
 **********
 **********
 **    2 **SET NOGRAD
 **********
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 f            5.00000e-01  1.00000e-01    0.00000e+00  1.00000e+00
     2 m            0.00000e+00  2.00000e+00   -1.00000e+01  1.00000e+01
     3 s            2.00000e+00  9.50000e-01    1.00000e-01  1.00000e+01
 **********
 **    3 **SET ERR         0.5
 **********
 **********
 **    4 **SET PRINT           1
 **********
 **********
 **    5 **SET STR           1
 **********
 NOW USING STRATEGY  1: TRY TO BALANCE SPEED AGAINST RELIABILITY
 **********
 **    6 **MIGRAD        1500           

Fit modelc with constraint term on parameter f

In [10]:
RooFitResult *r2 = modelc.fitTo(*d, Constrain(f), Save());

[#1] INFO:Minization --  Including the following contraint terms in minimization: (fconstraint)
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_modelc_modelData_with_constr) Summation contains a RooNLLVar, using its error level
[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
[#1] INFO:Minization --  The following expressions have been identified as constant and will be precalculated and cached: (poly)
[#1] INFO:Minization --  The following expressions will be evaluated in cache-and-track mode: (gauss)
 **********
 **   10 **SET PRINT           1
 **********
 **********
 **   11 **SET NOGRAD
 **********
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 f            6.49867e-01  1.21452e-01    0.00000e+00  1.00000e+00
     2 m            7.18243e-01  4.96642e-01   -1.00000e+01  1.00000e+01
     3 s            2.18801e+00  4.61289e-01    1.00000e-01  1.00000e+01
 **********
 **   12 **SET ERR         0.5
 ****

Method 2 -  specify external constraint when fitting
 -------------------------------------------------------------------------------------------------------

Construct another gaussian constraint p.d.f on parameter f at 0.2 with resolution of 0.1

In [11]:
RooGaussian fconstext("fconstext", "fconstext", f, RooConst(0.2), RooConst(0.1));

Fit with external constraint

In [12]:
RooFitResult *r3 = model.fitTo(*d, ExternalConstraints(fconstext), Save());

[#1] INFO:Minization --  Including the following contraint terms in minimization: (fconstext)
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_model_modelData_with_constr) Summation contains a RooNLLVar, using its error level
[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
[#1] INFO:Minization --  The following expressions have been identified as constant and will be precalculated and cached: (poly)
[#1] INFO:Minization --  The following expressions will be evaluated in cache-and-track mode: (gauss)
 **********
 **   19 **SET PRINT           1
 **********
 **********
 **   20 **SET NOGRAD
 **********
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 f            6.90394e-01  1.01490e-01    0.00000e+00  1.00000e+00
     2 m            6.95291e-01  4.98516e-01   -1.00000e+01  1.00000e+01
     3 s            2.26976e+00  4.75547e-01    1.00000e-01  1.00000e+01
 **********
 **   21 **SET ERR         0.5
 *******

Print the fit results

In [13]:
cout << "fit result without constraint (data generated at f=0.5)" << endl;
r1->Print("v");
cout << "fit result with internal constraint (data generated at f=0.5, constraint is f=0.8+/-0.2)" << endl;
r2->Print("v");
cout << "fit result with (another) external constraint (data generated at f=0.5, constraint is f=0.2+/-0.1)" << endl;
r3->Print("v");

fit result without constraint (data generated at f=0.5)

  RooFitResult: minimized FCN value: 134.849, estimated distance to minimum: 4.23315e-05
                covariance matrix quality: Full, accurate covariance matrix
                Status : MINIMIZE=0 HESSE=0 

    Floating Parameter  InitialValue    FinalValue +/-  Error     GblCorr.
  --------------------  ------------  --------------------------  --------
                     f    5.0000e-01    6.4987e-01 +/-  1.21e-01  <none>
                     m    0.0000e+00    7.1824e-01 +/-  4.97e-01  <none>
                     s    2.0000e+00    2.1880e+00 +/-  4.61e-01  <none>

fit result with internal constraint (data generated at f=0.5, constraint is f=0.8+/-0.2)

  RooFitResult: minimized FCN value: 134.19, estimated distance to minimum: 7.15613e-07
                covariance matrix quality: Full, accurate covariance matrix
                Status : MINIMIZE=0 HESSE=0 

    Floating Parameter  InitialValue    FinalValue +/-  Error 